Every Object-Oriented language has at least one standard way of getting a
string representation from any object.

Vector Class Redux
In order to demonstrate the many methods used to generate object
representations, we’ll use a Vector2d class similar to the one we saw in
Chapter 1.

Example 11-1. Vector2d instances have several representations

Example 11-2. vector2d_v0.py: methods so far are all special methods

In [48]:
from array import array
import math

class Vector2d:
    typecode = 'd'

    def __init__(self, x, y) -> None:
        self.__x = float(x)
        self.__y = float(y)

    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self) -> str:
        class_name = type(self).__name__
        return '{} ({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)
        
    def angle(self):
        return math.atan2(self.y, self.x)

    def __hash__(self):
        return hash(self.x) ^ hash(self.y)

In [34]:
v1 = Vector2d(3, 4)

In [35]:
v1

Vector2d (3.0, 4.0)

In [36]:
class Demo:
    @classmethod
    def klassmeth(*args):
        return args

    @staticmethod
    def statmeth(*args):
        return args

In [37]:
Demo.klassmeth()

(__main__.Demo,)

In [38]:
Demo.klassmeth('spam')

(__main__.Demo, 'spam')

In [39]:
Demo.statmeth()

()

In [40]:
Demo.statmeth('spam')

('spam',)

In [41]:
b = 1.486
format(b, '0.4f')

'1.4860'

In [42]:
'1 brl = {rate:0.2f} usd'.format(rate=b)

'1 brl = 1.49 usd'

In [43]:
f'1 br = {b:0.2f} usd'

'1 br = 1.49 usd'

Example 11-5 implements __format__ to produce the displays just
shown.
Example 11-5. Vector2d.format method, take #1

In [44]:
v1 = Vector2d(3, 4)

In [45]:
v1.__format__

<bound method Vector2d.__format__ of Vector2d (3.0, 4.0)>

To generate polar coordinates we already have the __abs__ method for
the magnitude, and we’ll code a simple angle method using the
math.atan2() function to get the angle. This is the code:

With that, we can enhance our __format__ to produce polar coordinates.
See Example 11-6.
Example 11-6. Vector2d.format method, take #2, now with polar
coordinates

In [46]:
format(Vector2d(1, 2), 'p')

'<2.23606797749979, 1.1071487177940904>'

In [47]:
format(Vector2d(1, 1), '0.5fp')

'<1.41421, 0.78540>'

A Hashable Vector2d
As defined, so far our Vector2d instances are unhashable, so we can’t put
them in a set:

Example 11-7. vector2d_v3.py: only the changes needed to make Vector2d
immutable are shown here; see full listing in Example 11-11

In [49]:
v1 = Vector2d(3, 4)
v2 = Vector2d(3.1, 4.2)

In [50]:
v2

Vector2d (3.1, 4.2)

In [51]:
hash(v1), hash(v2)

(7, 384307168202284039)